Period based on mass of central object:
$$T = 2\pi\sqrt{\frac{r^{3}}{GM}}$$
Lorentz factor for velocity:
$$\gamma_{velocity} = \frac{1}{\sqrt{1-(\frac{v}{c})^{2}}}$$ 

$\Delta t' = \Delta t\gamma_{v}$, Where **t'** is the elapsed time for an observer moving at speed **v**, and **t** is the elapsed time for a clock at rest.

Gravitational Time Dilation Formula
$$\gamma_{gravity} = \frac{1}{\sqrt{1-\frac{2GM}{rc^{2}}}} = \frac{1}{\sqrt{1-\frac{r_{s}}{c^{2}}}}$$

$\Delta t' = \Delta t\gamma_g$ Where **t'** is the elapsed time for an observer close to a gravitational object **t** is the elapsed time for an observer at a large distance.

Current problems in the code (first time using pygame):
1. Collisions with the event horizon result in imaginary numbers, crashing the program - currently this error is handled by setting them it to a high value when distance is shorter than s_radius.
2. Texts describing the values of distance, relative and inertial time, and the gamma factors for each planet currently print on top of each other when more than one planet is added to the list of planets.
3. There is no functionality for light - light acts as an object which can be slowed down by gravity instead of having a constant speed_magnitude of c
4. There is no functionality to measure length contraction between objects
5. There is no functionality to choose a frame of reference
6. There is no functionality allowing the user to set scale, timesteps, choose planets, change bh mass, set original velocities, etc
7. Objects inside the photon sphere can still have stable orbits
8. Pause button isn't displaying

In [4]:
# Creating a function to calculate escape velocity
def calc_escape_velocity(mass, distance):
    V_esc = np.sqrt((2 * G * mass)/distance)
    return V_esc

# Creating a function to find the distances of chosen escape velocities
def calc_esc_radius(mass, V_esc):
    # Should give swarzchild radius when V_esc = c
    d = (2 * G * mass)/V_esc **2
    return d

def calc_radial_velocity(radius, period):
    rad_vel = (2 * np.pi * radius)/period
    return rad_vel

def calc_radius_from_vel(rad_vel, period):
    # r = vT/2*pi
    radius = (rad_vel * period)/ (2 * np.pi)
    return radius

def calc_period(distance, mass):
    period = 2 * np.pi *(math.sqrt(distance**3/(G * mass)))
    return period

def calc_grav_time_dilation(s_radius, distance):
    if distance >= s_radius:
        grav_gamma      = 1/(1 - (s_radius / distance))**0.5
    else:
        grav_gamma = 1000 # avoiding imaginary numbers until I can get collisions to work
    return grav_gamma

def calc_vel_time_dilation(velocity):
    if velocity >= 0.9999 * c:
        vel_gamma = 1/(math.sqrt(1-(0.9999/c)**2))
        return vel_gamma
    else:
        vel_gamma = 1/(math.sqrt(1-(velocity/c)**2))
        return vel_gamma

# kg/m^3
def calc_bh_density(mass, s_radius):
    volume = (4/3)*(np.pi * s_radius **3)
    density = mass / volume
    return density

time_scales = [
    (1e9 * year, "billion year"),
    (1e6 * year, "million year"),
    (1e3 * year, "thousand year"),
    (year, "year"),
    (1, "second"),
    (1e-6, "microsecond"),
    (1e-9, "nanosecond"),
    (1e-12, "picosecond"),
    (5.39e-44, "Planck second")  
]

In [48]:
# Base units are in kg, m, and s
import numpy as np
import math

# Setting Constants
G          = 6.674e-11      # N * m^2 * kg^-2
c          = 2.99e+8        # m * s^-1
sol_mass   = 1.99e+30       # kg
earth_mass = 5.97e+24       # kg
moon_mass  = 7.348e+22      # kg
moon_dist  = 3.84e+8        # m
jup_mass   = 1.89e+27       # kg
au         = 1.49e+11       # m
year       = 3600*24*365.25 # s
second     = 1
lightyear  = c * year       # m

bh_mass    = 1e8*sol_mass # Intersteller gargantua = 1e8, TON618 = 6.6e10

moon_speed = calc_radial_velocity(moon_dist, 3600*24*27.32)
s_radius   = calc_esc_radius(bh_mass, c)
bh_density = calc_bh_density(bh_mass, s_radius)

n = 50 # shouldn't go below 1.5


orbit_n         =  n * s_radius
bh_period       = calc_period(orbit_n, bh_mass)
bh_orbit_speed  = calc_radial_velocity(orbit_n, bh_period)
escape_v_n      = calc_escape_velocity(bh_mass, orbit_n)
grav_gamma      = calc_grav_time_dilation(s_radius, orbit_n)
vel_gamma       = calc_vel_time_dilation(escape_v_n)
radial_gamma    = calc_vel_time_dilation(bh_orbit_speed)

# If the math is working right, the photon sphere should have an orbital speed of c
orbit_c         = calc_radial_velocity(1.5*s_radius, calc_period(1.5*s_radius,bh_mass))

if s_radius    <= 1e-3:
    print(f"Schwarzschild Radius = {s_radius:.2e}m")
else:
    print(f"Schwarzschild Radius = {s_radius:.2f}m or {s_radius/au:.2e}au")

print(f"Density of black hole = {bh_density:.3e}kg/m^3")

print(f"{n} time(s) the event horizon radius: {orbit_n:.2e}m, or {orbit_n/1000:.2e}km, or {orbit_n/au:.2e}au")

print(f"Period at {n} * event horizon: {bh_period:.2e}s or {bh_period/year:.2f}years.")

if bh_orbit_speed >= c/10:
    print(f"Planet is orbiting with relativistic speeds at {bh_orbit_speed/c:.2f}c")
else:
    print(f"Speed of orbiting planet: {bh_orbit_speed:.2e}m/s or {bh_orbit_speed/c:.2f}c")
    
print(f"Escape velocity at {n} schwarzchild radii: {escape_v_n:.2e}m/s or {escape_v_n/bh_orbit_speed:.2e} x radial speed or {escape_v_n/c:.2e}c.")

print(f"Gravitational gamma factor {grav_gamma}")

print(f"Escape Velocity gamma factor {vel_gamma:.2f}")

print(f"Radial orbit velocity gamma factor: {radial_gamma:.2f}")

if orbit_c == c:
    print("The photon sphere is accurately at 1.5 * the event horizon radius.")
else:
    print(f"The photon sphere is still broken - orbits are showing as stable at {orbit_c/c}c")


Schwarzschild Radius = 297116587062.78m or 1.99e+00au
Density of black hole = 1.811e+03kg/m^3
50 time(s) the event horizon radius: 1.49e+13m, or 1.49e+10km, or 9.97e+01au
Period at 50 * event horizon: 3.12e+06s or 0.10years.
Planet is orbiting with relativistic speeds at 0.10c
Escape velocity at 50 schwarzchild radii: 4.23e+07m/s or 1.41e+00 x radial speed or 1.41e-01c.
Gravitational gamma factor 1.0101525445522106
Escape Velocity gamma factor 1.01
Radial orbit velocity gamma factor: 1.01
The photon sphere is still broken - orbits are showing as stable at 0.5773502691896258c


In [50]:
import pygame
import sys
import math
pygame.init()

# Creating a figure and axis
width, height = 1280, 700
win = pygame.display.set_mode((width, height))
pygame.display.set_caption("Orbit Simulation")

# Color library
white        = (255,255,255)
black        = (0, 0, 0)
dark_gray    = (89, 89, 89)
yellow       = (255, 255, 0)
blue         = (0, 0, 255)
red          = (255, 0, 0)
cosmic_latte = (255, 248, 231) # average color of galaxies
gray         = (222, 222, 222)
light_blue   = (204,212,255)
brown        = (77,38,0)

# Font
font         = pygame.font.SysFont("ariel", 16)
pause_bttn   = pygame.Rect(20, 20, 100, 50)

# Time and space scaling
# Still needs to handle whether or not a black hole is the central object
# So that this can also be set for the solar system, just for fun
# For now, comment out the one not being used
dt            = bh_period/360
scale         = 300 / orbit_n

#dt            = year/36
#scale            = 10 / au

object_texts = {}

class Planet: 
    def __init__(self, x, y, radius, color, mass):
        self.x               = x
        self.y               = y
        self.radius          = radius # in pixels
        self.color           = color
        self.mass            = mass
        self.rel_clock       = 0
        self.intl_clock      = 0
        
        # Should also make light object functionality
        self.sun             = False
        self.distance_to_sun = 1 # Avoiding division by 0
        
        # To draw event horizons and handle collisions
        self.bh              = False
        self.absorbed        = False
        
        self.orbit           = []
        
        self.x_vel           = 0 # m/s
        self.y_vel           = 0 # m/s
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
            
            
    def draw_event_horizon(self, planets):
        for planet in planets:
            if self.bh:
                self.event_horizon = calc_esc_radius(self.mass, c)
                center        = (scale * self.x + (width/2), scale * self.y + (height/2))
                
                # Draw event horizon
                pygame.draw.circle(win, black, center, int(scale*self.event_horizon), 2)
                
                # Draw photon sphere
                pygame.draw.circle(win, yellow, center, int(scale * 1.5 * self.event_horizon),2)
    
    # This isn't working, I have no idea why not.    
    def distance_line(self, other_objects):
        self.position  = (self.x, self.y)
        for other in other_objects:
            other.position = (scale * other.x + (width/2), scale * other.y + (height/2))
            if other.sun or other.bh:
                pygame.draw.line(win, white, (self.position), (other.position), 1)

    
    def attraction(self, other):
        other_x, other_y = other.x, other.y
        distance_x       = other_x - self.x
        distance_y       = other_y - self.y
        distance         = math.sqrt(distance_x ** 2 + distance_y ** 2)
        
        if other.sun:
            self.distance_to_sun = distance
        elif other.bh:
            self.distance_to_sun = distance
        
        mu      = self.mass * other.mass
        force   = G * mu / distance ** 2
        theta   = math.atan2(distance_y, distance_x)
        force_x = math.cos(theta) * force
        force_y = math.sin(theta) * force
        return force_x, force_y
    
    def inertial_clock():
        inertial_clock += dt
        return inertial_clock
    
    def update_position(self, planets):
        # Initial forces at 0
        total_fx = total_fy = 0
        
        for planet in planets:
            # Avoiding division by 0
            if self == planet:
                continue
            fx, fy    = self.attraction(planet)
            total_fx += fx
            total_fy += fy
            
        # Calculate acceleration
        accel_x = total_fx / self.mass
        accel_y = total_fy / self.mass
    
        # Update velocity components
        self.x_vel += accel_x * dt
        self.y_vel += accel_y * dt
        
        self.x += self.x_vel * dt
        self.y += self.y_vel * dt
        
        self.orbit.append((self.x, self.y))
        
        self.speed_magnitude = math.sqrt(self.x_vel**2 + self.y_vel**2)
        if self.speed_magnitude >= c:
            # Cap velocity
            self.x_vel = self.x_vel / self.speed_magnitude * (c * 0.9999)
            self.y_vel = self.y_vel / self.speed_magnitude * (c * 0.9999)
    
    # Have things fall into the black hole
    def collide(self, other, planets):
        remove_from_list = False
        if not self.bh:
            if not self.sun:
                if other.bh:
                    if self.distance_to_sun <= other.event_horizon +1:
                        other.mass += self.mass
                        #self.absorbed = True
                        self.remove_from_list = True
                        
    
    def draw(self, win):
        x           = self.x * scale + width/2
        y           = self.y * scale + height/2
        
        
        if len(self.orbit) > 2:
            # Keep only the most recent 1000 points
            if len(self.orbit) > 30000:
                self.orbit = self.orbit[-30000:]
            
            updated_points = []
            
            for point in self.orbit:
                x, y = point
                x = x * scale + width/2
                y = y * scale + height/2
                updated_points.append((x,y))

            pygame.draw.lines(win, self.color, False, updated_points, 1)
         
        pygame.draw.circle(win, self.color, (x,y), self.radius)
        
        if not self.sun:
            if not self.bh:
                self.grav_gamma     = calc_grav_time_dilation(s_radius, self.distance_to_sun)

                grav_gamma_text     = font.render(f"Gγ: {self.grav_gamma:.3f}", 1, self.color)

                if self.speed_magnitude <= 0.9999*c:
                    self.vel_gamma  = calc_vel_time_dilation(math.sqrt(self.x_vel**2 + self.y_vel**2))
                else:
                    self.vel_gamma  = 70.1

                vel_gamma_text      = font.render(f"Vγ: {self.vel_gamma:.3f}", 1, self.color)

                # Update inertial and relative time counters
                self.rel_clock     += dt / (self.vel_gamma * self.grav_gamma)
                self.intl_clock    += dt

                time_diff           = self.intl_clock - self.rel_clock

                if self.speed_magnitude <= c/1000:
                    vel_text        = font.render(f"Velocity of planet: {self.speed_magnitude:.3f}m/s",1, self.color)
                else:
                    vel_text        = font.render(f"Velocity of planet: {self.speed_magnitude/c:.3f}c",1, self.color)

                # Change the time scale as the counters gets higher
                for exponent, unit in time_scales:
                    # Check if time_counter or relative_time is greater than the current scale
                    if self.rel_clock >= exponent:
                        # Use the current scale for rendering
                        if unit == "Planck second":
                            rel_time_text   = font.render(f"Time in {unit}s: {self.rel_clock/exponent:.3e}", 1, self.color)
                            time_diff_text  = font.render(f"Difference in times: {time_diff/exponent:.3e} {unit}s", 1, self.color)
                            intl_clock_text = font.render(f"Distant observer clock in {unit}s: {self.intl_clock/exponent:.3e} {unit}s", 1, self.color)
                        else:
                            rel_time_text   = font.render(f"Clock on planet in {unit}s: {self.rel_clock/exponent:.3f}", 1, self.color)
                            time_diff_text  = font.render(f"Difference in times: {time_diff/exponent:.3f} {unit}s", 1, self.color)
                            intl_clock_text = font.render(f"Distant observer clock in {unit}s: {self.intl_clock/exponent:.3f} {unit}s", 1, self.color)
                        break

                #print(f"Debugging: dt: {dt:.3f} Vγ: {self.vel_gamma:.3f}, Gγ: {grav_gamma:.3f}, Rel_clock: {self.rel_clock:.5f})")#, Inertial clock: {self.intl_clock:.5f}")

                if self.distance_to_sun >= lightyear:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius)/lightyear:.2f}lyrs from EH", 1, self.color)
                elif self.distance_to_sun >= au/1000:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius)/au:.3e}au from EH", 1, self.color)
                else:
                    distance_text = font.render(f"Planet is {(self.distance_to_sun-s_radius):.3e}m from EH", 1, self.color)

                object_texts[self] = {
                    "intl_clock": intl_clock_text,
                    "rel_time": rel_time_text,
                    "time_diff": time_diff_text,
                    "distance": distance_text,
                    "vel": vel_text,
                    "grav_gamma": grav_gamma_text,
                    "vel_gamma": vel_gamma_text
                }

            for obj, texts in object_texts.items():
                y_offset = 10
                for text_name, text_surface in texts.items():
                    win.blit(text_surface, (10, y_offset))
                    y_offset += 20
            
             
def main():
    run            = True
    paused         = False
    clock          = pygame.time.Clock()
    time_counter   = 0
    relative_time  = 0
    time_diff      = 0
    
    sun            = Planet(0,0, 9, yellow, sol_mass)
    sun.sun        = True
    
    black_hole     = Planet(0,0,3, black, bh_mass)
    black_hole.sun = True
    black_hole.bh  = True
    
    mercury        = Planet(0.39 * au, 0, 2, white, 0.33e+24)
    mercury.y_vel  = -calc_radial_velocity(0.39*au, 0.24*year)
    
    venus          = Planet(0.72 * au, 0, 3, brown, 4.87e+24)
    venus.y_vel    = -calc_radial_velocity(0.72*au, 0.62*year)
    
    earth          = Planet(au, 0, 3, blue, earth_mass)
    earth.y_vel    = -calc_radial_velocity(au, year)
    
    moon           = Planet(au + moon_dist, 0, 1, gray, moon_mass)
    moon.y_vel     = -(moon_speed-earth.y_vel)
    
    mars           = Planet(1.524 * au, 0, 2, red, 0.11 * earth_mass)
    mars.y_vel     = -2.41e4
    
    jupiter        = Planet(-5.2 * au, 0, 5, red, jup_mass)
    jupiter.y_vel  = calc_radial_velocity(5.2 * au, 12* year)
    
    saturn         = Planet(-9.54 * au,0, 4, yellow, 568e+24)
    saturn.y_vel   = calc_radial_velocity(9.54 * au, 28.7*year)
    
    uranus         = Planet(-19.2*au, 0, 4, blue,8.68e25)
    uranus.y_vel   = calc_radial_velocity(19.2*au, 2.65e9) # had the value in seconds
    
    neptune        = Planet(-30.1*au, 0, 4, blue, 1.02e26)
    neptune.y_vel  = calc_radial_velocity(30.1*au, 5.2e9)
    
    #light          = Planet(1.5 * s_radius,0, 1, white, 1)
    #light.y_vel    = -np.sin(c)
    #light.x_vel    = -np.cos(c)
    
    t_bh           = Planet(-orbit_n, 0, 2, black, bh_mass/2)
    t_bh.y_vel     = -0.8*bh_orbit_speed
    t_bh.bh        = True
    t_bh.sun       = True
    
    t_plnt         = Planet(-orbit_n, 0, 2, white, bh_mass/1000)
    t_plnt.y_vel   = -bh_orbit_speed
    
    # Make the differences from t_plnt a single variable
    t2 = 1.2
    t2_plnt        = Planet(t2 * orbit_n, 0, 2, white, 0.1*bh_mass)
    t2_plnt.y_vel  = 0.8*calc_radial_velocity(t2 * orbit_n, calc_period(t2*orbit_n, bh_mass + t_plnt.mass))
    
    t3 = 1.5
    t3_plnt        = Planet(-t3 * orbit_n, 0, 2, white, 0.2*bh_mass)
    t3_plnt.y_vel  = -1.1*calc_radial_velocity(t3 * orbit_n, calc_period(t3*orbit_n, bh_mass + t_plnt.mass + t2_plnt.mass))
    
    t4 = 2
    t4_plnt        = Planet(t4 * orbit_n, 0, 2, white, moon_mass)
    t4_plnt.y_vel  = 0.6*calc_radial_velocity(t4*orbit_n, calc_period(t4 * orbit_n, bh_mass+t_plnt.mass+t2_plnt.mass+t3_plnt.mass))
    
    planets = [black_hole, t_bh]
    
    while run:
        clock.tick(60)
        win.fill(dark_gray)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if pause_bttn.collidepoint(event.pos):
                        paused = not paused
        if not paused:
            # Change the time scale as the counters gets higher
            for exponent, unit in time_scales:
                # Check if time_counter or relative_time is greater than the current scale
                if time_counter >= exponent:
                    # Use the current scale for rendering
                    if unit == "Planck second":
                        time_text     = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3e}", 1, white)
                    else:
                        time_text = font.render(f"Inertial Clock in {unit}s: {time_counter/exponent:.3f}", 1, white)
                    break

            for planet in planets:
                planet.update_position(planets)
                planet.draw(win)
                planet.draw_event_horizon(planets)
                #if planet.collide(black_hole, planets):
                #    if hasattr(planet, 'remove_from_list') and planet.remove_from_list:
                #        planets.remove(planet)

    

            
        pygame.display.update()
                
    pygame.quit
    
    
main()


SystemExit: 